## Implementacja populacji
- `chromoBits` - liczba bitów chromosomów osobników populacji, liczba całkowita
- `size` - liczba osobnikow w populacji, liczba całkowita

#### bestSelection - implementacja selekcji najlepszych
- `percent` - procent najlepszych osobników, liczba całkowita
- `newPopulation` - najlepiej przystostowane osobniki z populacji, nowa populacja osobników

#### tournamentSelection - implementacja selekcji turniejowej
- `k` -  wielkość turnieju, liczba całkowita
- `pop.individuals[0]` - osobnik wybrany z populacji selekcją turniejową, obiekt klasy Individual

#### rouletteWheelSelection - implementacja selekcji kołem ruletki
- `self.individuals[bestIndex]` - osobnik wybrany z populacji selekcją kołem ruletki, obiekt klasy Individual

##### Dodatkowo: calculateDistribution - implementacja wyznaczenia dystrybucji osobników dla danej populacji
   - `temp` - dystrybucje osobników z populacji, lista


In [1339]:
class Population:
    def __init__(self, size, chromoBits):
        self.size = size
        self.chromoBits = chromoBits
        if size > 0:
            self.individuals = self.setPopulation()
        else :
            self.individuals = []
        
    def setPopulation(self):    
        return [Individual(self.chromoBits) for row in range(self.size)]
    
    def addIndividual(self, ind):    
        self.individuals.append(ind)
        self.size += 1
        
    def bestSelection(self, percent = 30):
        popAdaptation = []
        bestSelected = []
        N = int(self.size * percent / 100)

        for ind in self.individuals:
            popAdaptation.append(ind.boothFunc())  

        for i in range(0, N):  
            best = max(popAdaptation)
            bestSelected.append(best)
            popAdaptation.remove(best) 

        newPopulation = Population(0, 8)
        for best in bestSelected:
            for ind in self.individuals:
                if ind.boothFunc() == best:
                    newPopulation.addIndividual(ind)
        return newPopulation
    
    
    def tournamentSelection(self, k):
        pop = copy.deepcopy(self)    
        popSize = len(pop.individuals)

        if popSize == 1:
            return pop.individuals[0]
        else:
            best = Population(0, self.chromoBits)
            # create tournaments using all individuals from population
            while(popSize > 0):
                popSize = len(pop.individuals)
                tournament = Population(0, self.chromoBits)

                # select k random individuals from population to new tournament (without repetition)
                if popSize >= k:
                    for i in range(k):
                        randInd = random.choice(pop.individuals)
                        tournament.addIndividual(randInd)
                        pop.individuals.remove(randInd)
                elif popSize > 1:
                    tournament = copy.deepcopy(pop)    
                    pop.individuals.clear() 
                else:
                    break

                # select the best individuals tournament (stochastic tournament selection)
                best.addIndividual(random.choice(tournament.individuals)) 

            return tournamentSelection(best, k)
        
    def calculateDistribution(self):
        temp = [] 
        # adaptation function
        for ind in self.individuals:
            temp.append(ind.reciprocalBoothFunc())
        adaptationSum = sum(temp)
        # calculate every individual probability
        temp = [adaptation/adaptationSum for adaptation in temp] 
        # calculate every individual probability
        for i in range(1, len(temp)):
            temp[i] += temp[i-1]
        return temp
    
    
    def rouletteWheelSelection(self):
        populationDist = self.calculateDistribution()
        self.showRouletteWheel(populationDist)
        # spin roulette wheel and find the best individual
        rand = random.random()
        previous = 0
        bestIndex = -1
        for index, value in enumerate(populationDist):
            if(previous <= rand <= value):
                bestIndex = index
            previous = value
        return self.individuals[bestIndex], fig

##### showRouletteWheel - implementacja prezentacji koła ruletki dla populacji
   - `individuals` - lista osobników, lista obiektów typu Individual
   - `populationDist` - dystrybucje osobników, lista 

In [1355]:
import plotly.graph_objects as go

def showRouletteWheel(individuals, populationDist):
    stringPop = []
    for ind in individuals:
        stringPop.append(str(ind.X.binary) + str(ind.Y.binary))
    fig = go.Figure(data = [go.Pie(labels = stringPop, values = populationDist)])
    fig.show()

In [1356]:
pop = Population(20,8)
dist = pop.calculateDistribution()
showRouletteWheel(pop.individuals, dist)